In [ ]:
import pandas as pd
samples_path = '../logs/samples.csv'
samples = pd.read_csv(samples_path, index_col=0)
samples = [(s[0:25].tolist(), s[25:].tolist()) for s in samples.values][0:1]

In [ ]:
from sensitivity import test_sensitivity

nnet_path = '../network/models/v3.2.2/model.nnet'
sym_results = test_sensitivity(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, save_results=True, save_samples=True, verbose=2, outdir='../logs/test/sensitivity/symmetric')
asym_results = test_sensitivity(nnet_path, samples, e_min=0.0001, e_max=100, e_prec=0.00001, asym=True, save_results=True, save_samples=True, verbose=2, outdir='../logs/test/sensitivity/asymmetric')

print('Symmetric results: ', sym_results)
print('Asymmetric results: ', asym_results)

In [ ]:
from sensitivity import check_sensitivity

sym_check_results = check_sensitivity(nnet_path, samples, sym_results)
asym_check_results = check_sensitivity(nnet_path, samples, asym_results, asym=True)

print('Symmetric check results:', sym_check_results)
print('Asymmetric check results:', asym_check_results)

In [ ]:
from sensitivity import test_sensitivity
from clustering import LGKMeansUtils

nnet_path = '../network/models/latest/model.nnet'
n_outputs = 5
lgkm = LGKMeansUtils.load('../artifacts/lgkm.pkl')
slow_regions = lgkm.get_regions(category=4, sort=True)
region = slow_regions[0] 
s = (region.centroid.tolist(), [1 if y==region.category else 0 for y in range(n_outputs)])

sym_results = test_sensitivity(nnet_path, [s], e_min=0.001, e_max=region.radius, e_prec=0.0001, verbose=2)
print('Symmetric results: ', sym_results)

In [ ]:
from sensitivity import test_sensitivity
from clustering import LGKMeansUtils

nnet_path = '../network/models/latest/model.nnet'
n_outputs = 5
lgkm = LGKMeansUtils.load('../artifacts/lgkm.pkl')
slow_regions = [r for r in lgkm.get_regions(category=4, sort=True) if r.n > 1]
slow_results = []
for r in slow_regions:
    s = (r.centroid.tolist(), [1 if y==r.category else 0 for y in range(n_outputs)])
    result = test_sensitivity(nnet_path, [s], asym=True, e_min=0.001, e_max=r.radii.max(), e_prec=0.0001, verbose=2)
    slow_results.append(result)
print('Slow Symmetric results complete.')

In [ ]:
# slow_results